Кстати, может кому то все еще интересно посмотреть как решался вступительный контест.
Выкладываю ноутбук для топ-1 решения. Соло модель без блендов/стэкингов и т.п.
Насколько я понял, там была вся суть в природе данных - скорее всего это была выгрузка из какой то рекомендательной системы или типа того. то есть, самые важные фичи - таймдельты между записями - т.к. если система чето показывает пользователю, то скорее всего он еще не выбрал статью которую прочитать (собственно это лик)

In [ ]:
import pandas as pd
import numpy as np
import tqdm
from itertools import combinations
import gc
import lightgbm as lgb

In [ ]:
df = pd.read_csv('./materials/train.tar.gz', compression='gzip', header=0, sep=';', quotechar='"')
df = df[:-1]
df_test = pd.read_csv('./materials/test-data.tar.gz', compression='gzip',
                      header=0, sep=',', quotechar='"', index_col=0)
df.columns = ['time', 'label', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
       'C9', 'C10', 'CG1', 'CG2', 'CG3', 'l1', 'l2', 'C11', 'C12']
df_test.columns = ['time', 'label', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
       'C9', 'C10', 'CG1', 'CG2', 'CG3', 'l1', 'l2', 'C11', 'C12']

In [ ]:
full_dataset = pd.concat([df, df_test])
full_dataset['time'] = pd.to_datetime(full_dataset['time'], unit='s')

In [ ]:
del df, df_test

In [ ]:
calc_cols = list(full_dataset.columns.difference(['time', 'label']))+ [('C1', 'C2'),('C1', 'C5'),('C1', 'C8'),
                         ('C5', 'C2'), ('C8', 'C2'), ('C1', 'C2', 'C5'),
                        ('C1', 'C2', 'C8')
                        ]

In [ ]:
for col in tqdm.tqdm(calc_cols):
    a = full_dataset.groupby(col)['time'].shift(1)
    b = full_dataset.groupby(col)['time'].shift(-1)
    c = full_dataset.groupby(col)['time'].transform('max')
    d = full_dataset.groupby(col)['time'].transform('min')
    
    
    full_dataset['from_prev_{}'.format(col)] = (full_dataset['time'] - a).dt.total_seconds()
    full_dataset['to_next_{}'.format(col)] = (b - full_dataset['time']).dt.total_seconds()
    
    full_dataset['from_max_{}'.format(col)] = (full_dataset['time'] - d).dt.total_seconds()
    full_dataset['to_min_{}'.format(col)] = (c - full_dataset['time']).dt.total_seconds()
    

In [ ]:
full_dataset['hour'] = full_dataset['time'].dt.hour

In [ ]:
X_train = full_dataset[full_dataset['label'].isin([0, 1])]
X_test = full_dataset[full_dataset['label']==-1]

In [ ]:
good_cols = list(X_train.columns.difference(['time',
                                                 'CG1',
                                                 'CG2',
                                                 'CG3',
                                                 'label']))

In [ ]:
target_feature = 'label'

In [ ]:
D_train = lgb.Dataset(
    X_train[good_cols],
    X_train[target_feature]
                      )

In [ ]:
px = {
        'boost_from_average':'false',
        'learning_rate': 0.01,
        'max_depth': -1,
        'min_data_in_leaf': 256,
        'num_threads': 12,
        'tree_learner': 'serial',
    'num_leaves':128,
    'max_bin':254,
        'verbosity': 1,
    'objective':'binary',
    'metric':'binary'
    }
lg_0 = lgb.train(px, D_train,
                num_boost_round=5100)

In [ ]:
y_pred = lg_0.predict(X_test[good_cols])

In [ ]:
pd.Series(y_pred).to_csv('final_subm.csv', index=False, header=False)